In [0]:
%pip install transformers python-doctr==0.10.0 deepdoctection[pt]
%restart_python

In [0]:
import deepdoctection as dd

def process_document(document_path):
    doc_data_return = []
    analyzer = dd.get_dd_analyzer()
    df = analyzer.analyze(path = document_path)
    df.reset_state()
    doc_iter = iter(df)
    page_num = 0
    while True:
        try:
            page = next(doc_iter)
            page_num += 1
            page_text = page.text
            tables_html = [table.html for table in page.tables]
            
            doc_data_return.append({
                'doc_id': document_path,
                'doc_name': document_path.split('/')[-1],
                'page_num': page_num,
                'page_text': page.text,
                'tables_html': tables_html,
            })
            break
        except StopIteration:
            print(f"last page processed {page_num}")
            break


    return doc_data_return

In [0]:
import os

file_path = "/Volumes/pdf_parsing/test_parsing/raw_files/aph_example_1.pdf"
data_returned = process_document(file_path)

In [0]:
data_returned

In [0]:
volume_path = "/Volumes/pdf_parsing/test_parsing/raw_files"
os.listdir(volume_path)


In [0]:
import os

volume_path = "/Volumes/pdf_parsing/test_parsing/raw_files"
all_doc_data = []

for filename in os.listdir(volume_path):
    file_path = os.path.join(volume_path, filename)
    if os.path.isfile(file_path):
        doc_data = process_document(file_path)
        all_doc_data.extend(doc_data)

In [0]:
df = spark.createDataFrame(all_doc_data)
df.write.mode("overwrite").saveAsTable("pdf_parsing.test_parsing.doc_parsed_data")

In [0]:
analyzer = dd.get_dd_analyzer()
df = analyzer.analyze(path = "/Volumes/pdf_parsing/test_parsing/raw_files/aph_example_1.pdf")

In [0]:
df.reset_state()
doc = iter(df)

In [0]:
pagezz = next(doc)